#Converting user comments data into SQL table

In [29]:
import sys
sys.path.append('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')
import redditDataIO
import pandas as pd
import numpy as np
import os
os.chdir('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')

Loading the data from a csv:

In [5]:
userCommentData = pd.read_csv('Data/6.20.15-UserComment.csv')

/Users/alexsutherland/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1139: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Let's look to see if there's anything we can clean up:

In [6]:
userCommentData.head()

,Unnamed: 0,score,controversiality,body,subreddit,link_title,link_id,created_utc,username,userid
0,0,2,0,Lenny is the man. I'm good friends with his so...,funny,Millionaires In America (Not OC),t3_2u8mg2,1422674804,BrovietUnion1,1
1,1,11,0,"CondoInternet, brobeans.",technology,SpaceX confirms $1 billion funding round with ...,t3_2t3giy,1421824107,BrovietUnion1,1
2,2,2,0,"Baller, thanks!! Happy new year!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420078827,BrovietUnion1,1
3,3,1,0,"Awesome, thanks for the advice!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420048144,BrovietUnion1,1
4,4,3,0,Great call! I'm from a place with 0 public tra...,Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420008479,BrovietUnion1,1


The first column is not useful, let's drop that. 

In [11]:
userCommentDataCleaned = userCommentData.drop(['Unnamed: 0'],axis=1)
userCommentDataCleaned.head()

,score,controversiality,body,subreddit,link_title,link_id,created_utc,username,userid
0,2,0,Lenny is the man. I'm good friends with his so...,funny,Millionaires In America (Not OC),t3_2u8mg2,1422674804,BrovietUnion1,1
1,11,0,"CondoInternet, brobeans.",technology,SpaceX confirms $1 billion funding round with ...,t3_2t3giy,1421824107,BrovietUnion1,1
2,2,0,"Baller, thanks!! Happy new year!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420078827,BrovietUnion1,1
3,1,0,"Awesome, thanks for the advice!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420048144,BrovietUnion1,1
4,3,0,Great call! I'm from a place with 0 public tra...,Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420008479,BrovietUnion1,1


In [15]:
userCommentDataCleaned.describe()

,created_utc,userid
count,9.734580e+05,973458.000000
mean,1.394449e+09,1557.345064
std,3.160666e+07,958.313796
min,1.156315e+09,1.000000
25%,1.375850e+09,748.000000
50%,1.400912e+09,1514.000000
75%,1.420354e+09,2350.000000
max,1.434830e+09,3341.000000


For some reason, the score and controversiality columns are now showing up in the .describe() table. Let's investigate why:

In [14]:
userCommentDataCleaned['score'].value_counts()

1     351474
2     172968
3      80593
1      53513
4      29696
0      28278
5      27079
2      26779
6      19845
7      14491
3      12879
8      11082
9       8802
-1      8235
10      7142
...
2175     1
1050     1
1053     1
1055     1
1057     1
2370     1
664      1
765      1
trees    1
3544     1
768      1
761      1
767      1
764      1
1054     1
Length: 2057, dtype: int64

It looks like there is some text in this column:

In [16]:
userCommentDataCleaned.ix[userCommentDataCleaned['score'] == 'trees',:]

,score,controversiality,body,subreddit,link_title,link_id,created_utc,username,userid
374706,trees,My smoke spot for tonight. Go Avs!,t3_24f1fg,1398921541.0,JimmyTree46,1160,NaN,NaN,NaN


Let's drop this row to see if that improves things:

In [17]:
userCommentDataCleaned = userCommentDataCleaned.drop(374706)

In [51]:
userCommentDataCleaned2 = userCommentDataCleaned.copy()
for item in userCommentDataCleaned['score'].value_counts().index:
    try:
        int(item)
    except:
        userCommentDataCleaned2 = userCommentDataCleaned2.drop(userCommentDataCleaned2.ix[userCommentDataCleaned2['score'] == item].index.values)

In [52]:
userCommentDataCleaned.shape

(973795, 9)

In [53]:
userCommentDataCleaned2.shape

(973690, 9)

In [55]:
userCommentDataCleaned2.dtypes

score                object
controversiality     object
body                 object
subreddit            object
link_title           object
link_id              object
created_utc         float64
username             object
userid              float64
dtype: object

It still thinks score is an object, not an int/float. Let's try to convert this:

In [58]:
userCommentDataCleaned2['score'] = userCommentDataCleaned2['score'].convert_objects(convert_numeric=True)

In [59]:
userCommentDataCleaned2.dtypes

score               float64
controversiality     object
body                 object
subreddit            object
link_title           object
link_id              object
created_utc         float64
username             object
userid              float64
dtype: object

In [60]:
userCommentDataCleaned2.describe()

,score,created_utc,userid
count,973564.000000,9.734580e+05,973458.000000
mean,6.809339,1.394449e+09,1557.345064
std,53.149711,3.160666e+07,958.313796
min,-245.000000,1.156315e+09,1.000000
25%,1.000000,1.375850e+09,748.000000
50%,2.000000,1.400912e+09,1514.000000
75%,3.000000,1.420354e+09,2350.000000
max,6014.000000,1.434830e+09,3341.000000


Great, score is now a 64 bit float. Let's look into why controversiality is not a float:

In [61]:
userCommentDataCleaned2['controversiality'].value_counts()

0    827897
0    128658
1     14595
1      2414
dtype: int64

Looks like an issue with some being ints and others being strings, let's do another convert objects:

In [62]:
userCommentDataCleaned2['controversiality'] = userCommentDataCleaned2['controversiality'].convert_objects(convert_numeric=True)

In [63]:
userCommentDataCleaned2.describe()

,score,controversiality,created_utc,userid
count,973564.000000,973564.000000,9.734580e+05,973458.000000
mean,6.809339,0.017471,1.394449e+09,1557.345064
std,53.149711,0.131018,3.160666e+07,958.313796
min,-245.000000,0.000000,1.156315e+09,1.000000
25%,1.000000,0.000000,1.375850e+09,748.000000
50%,2.000000,0.000000,1.400912e+09,1514.000000
75%,3.000000,0.000000,1.420354e+09,2350.000000
max,6014.000000,1.000000,1.434830e+09,3341.000000


Looks good. Let's look at the full table again:

In [65]:
userCommentDataCleaned2

,score,controversiality,body,subreddit,link_title,link_id,created_utc,username,userid
0,2,0,Lenny is the man. I'm good friends with his so...,funny,Millionaires In America (Not OC),t3_2u8mg2,1422674804,BrovietUnion1,1
1,11,0,"CondoInternet, brobeans.",technology,SpaceX confirms $1 billion funding round with ...,t3_2t3giy,1421824107,BrovietUnion1,1
2,2,0,"Baller, thanks!! Happy new year!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420078827,BrovietUnion1,1
3,1,0,"Awesome, thanks for the advice!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420048144,BrovietUnion1,1
4,3,0,Great call! I'm from a place with 0 public tra...,Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420008479,BrovietUnion1,1
5,1,0,That number was from when I was cutting 2-3 lb...,progresspics,"Update [M/24/6'0""]: 1.5 years later, got fat a...",t3_2owbfv,1418377352,BrovietUnion1,1
6,2,0,Haha well thanks for the ideas. This thread wa...,AskWomen,"I've often wondered: What is your perfect ""fir...",t3_2p0cyo,1418345620,BrovietUnion1,1
7,2,0,Sounds super fun! I'd definitely be down haha ...,AskWomen,"I've often wondered: What is your perfect ""fir...",t3_2p0cyo,1418344135,BrovietUnion1,1
8,2,0,I credit hot broga,progresspics,"Update [M/24/6'0""]: 1.5 years later, got fat a...",t3_2owbfv,1418335103,BrovietUnion1,1
9,1,0,"I like the ""no table separating you"" bit. I ne...",AskWomen,"I've often wondered: What is your perfect ""fir...",t3_2p0cyo,1418334609,BrovietUnion1,1


In [66]:
userCommentDataCleaned2['subreddit'].value_counts()

AskReddit          107219
progresspics        35190
loseit              29264
funny               25373
pics                24968
AdviceAnimals       20350
Fitness             18635
keto                15031
WTF                 14239
MakeupAddiction     12146
videos              10504
gaming              10200
trees                8090
IAmA                 7942
nfl                  7558
...
mildlyenteristing        1
ecchi                    1
nerdblock                1
RandomActsOfPetFood      1
cfbball                  1
tmbg                     1
cheerleaders             1
OregonCoast              1
TechnicalDeathMetal      1
Sum41                    1
Techno                   1
thronescomics            1
colorador4r              1
aaaaaatheismmmmmmmmmm    1
StarWarsEU               1
Length: 9351, dtype: int64

Alright, let's output this into a sql table:

In [67]:
redditDataIO.exportDataToSQL(userCommentDataCleaned2, tableName='userCommentData')

Index([u'score', u'controversiality', u'body', u'subreddit', u'link_title', u'link_id', u'created_utc', u'username', u'userid'], dtype='object')


/Users/alexsutherland/anaconda/lib/python2.7/site-packages/pymysql/cursors.py:134: Warning: Unknown table 'redditweightdatabase.usercommentdata'
  result = self._query(query)


InternalError: (InternalError) (1054, u"Unknown column 'nan' in 'field list'") 'INSERT INTO `userCommentData` (`index`, score, controversiality, body, subreddit, link_title, link_id, created_utc, username, userid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)' ((0, 2.0, 0.0, "Lenny is the man. I'm good friends with his son and met him on one occasion; he loves his role as batman and brings a lot of happiness to people under the radar. The guy is a legend in my circle of friends.", 'funny', 'Millionaires In America (Not OC)', 't3_2u8mg2', 1422674804.0, 'BrovietUnion1', 1.0), (1, 11.0, 0.0, 'CondoInternet, brobeans.', 'technology', 'SpaceX confirms $1 billion funding round with Google', 't3_2t3giy', 1421824107.0, 'BrovietUnion1', 1.0), (2, 2.0, 0.0, 'Baller, thanks!! Happy new year!', 'Seattle', "First New Year's Eve in Seattle, and I have a friend arriving at the airport at 8:30pm on NYE. Is it better for me to pick my friend up, or have them take a cab to me? Any tips/advice?", 't3_2

Whoops, we need to remove any row that has NaNs

In [68]:
userCommentDataCleaned3 = userCommentDataCleaned2.dropna()

In [71]:
userCommentDataCleaned3.shape[0] - userCommentDataCleaned2.shape[0]

-251

That removed 251 rows that had NaN. With those gone, let's try exporting again:

In [73]:
redditDataIO.exportDataToSQL(userCommentDataCleaned3, tableName='userCommentData')

Index([u'score', u'controversiality', u'body', u'subreddit', u'link_title', u'link_id', u'created_utc', u'username', u'userid'], dtype='object')
Exported Data to SQL Table (Name: userCommentData)
